### St&auml;dtedaten putzen
Probleme:
- Mehrfaches Vorkommen des gleichen Namen
- Welche Namen verwenden? 
  Deutsch, englisch, franz&ouml;sisch, italienisch, r&auml;toromanisch?
- Umlaute, Akzente
- Tippfehler, 


Vorgehen:
- `citynames`: Liste der Ortsnamen aus dem Original
- `dictlist`: zugeh&ouml;rige Liste von Dictionaries
- `name_idxs`: Dictionary der Form `{<name>: [<index1>,..], ...}`.  
  Verschiedene Ortschaften haben zum Teil den gleichen Namen.
- `name_idx`: Dictionary der Form `{<name>: <index>, ...}`.  
  Verschiedene Namen k&ouml;nnen auf den gleichen Index verweisen, 
  bew. die gleich Ortschaft bezeichnen.   
  
  Zum Beispiel hat `name_idx['Zuerich']` und
  `name_idx['Zürich']` den Wert `0`,
  und `dictlist[0]` den Wert `{'lat': 47.3786, 'lng': 8.54, 'population': 434008}`.

***
M&ouml;gliche Probleme mit Umlauten und Akzenten:  
Verschienene Strings repr&auml;sentieren das gleiche Zeichen!  
Schl&uuml;ssel nicht eindeutig.
***

In [ ]:
accent = chr(776)
umlaute1 = [ch + accent for ch in 'AOU']
umlaute2 = [chr(196), chr(214), chr(220)]
for x,y in zip(umlaute1, umlaute2):
    print('{} == {} is {},'.format(x,y,x==y),
          'len({})={}'.format(x,len(x)),
          'len({})={}'.format(y,len(y)),
         )

In [ ]:
'Ä' == 'Ä', len('Ä'), len('Ä')

***
.csv Daten einlesen
***

In [ ]:
from toolbox import table2str

lines = []
with open('/home/jovyan/work/NIA22Prog/data/ch.csv', mode ='r') as f:
    for line in f:
        lines.append(line.rstrip().split(','))
        
fields = (0,1,2,5,7)
sample = lines[:2] +  lines[-1:]     
sample = []
for line in lines[:3] +  lines[-2:]:
    sample.append([line[i] for i in fields])
    
print(table2str(sample[0], sample[1:]))

***
Kantone (admin_name) herausgreifen und sortieren.  
Gleiche Ortschaftsnamen werden durch Anh&auml;ngen des Kantonsk&uuml;rzels eindeutig gemacht.
***

In [ ]:
kt_set = set(line[5] for line in lines[1:]) # Kantone als Menge, 
                                            # jedes Element tritt genau einmal auf
# Menge sortieren -> Liste    
kt = sorted(kt_set)
len(kt), kt[:2] + kt[-2:]

***
Dictionary `{<Kantonskürzel>: <Kanton>,...}` erstellen
***

In [ ]:
kk = ['AG', 'AR', 'AI', 'BL', 'BS', 'BE', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 
      'NW', 'OW', 'SG', 'SH', 'SZ', 'SO', 'TG', 'TI', 'UR', 'VS', 'VD', 'ZG', 'ZH',
     ]
kt_kk = dict(zip(kt, kk))
{kt:kk for i,(kt,kk) in enumerate(kt_kk.items()) if i < 2 or i > 23}

***
Wir erstellen eine Liste `citynames` mit den Ortschaftsnamen,
eine zugeh&ouml;rige Liste von Dictionaries mit Schl&uuml;ssel 'lat', 'lng' und allenfalls 'population', sowie einen Dictionary `name_idxs`, der jedem Ortschaftsnamen seine Position(en) (Index) in der Liste `citynames` zuordnet.
***

In [ ]:
# Zu extrahierende Felder, gewuenschter Typ
fields = {1:float, 2:float, 7: int}

citynames = [] 
dictlist  = []
name_idxs = {}

for i,row in enumerate(lines[1:]):
    # key nur hinzufuegen, falls row[i] nicht der leere String ist
    city = row[0]
    data = {lines[0][i]: f(row[i]) for i,f in fields.items() if row[i]}
    
    citynames.append(city)
    dictlist.append(data)
    name_idxs.setdefault(city.upper(),[]).append(i)

***
Mehrfach auftretende Ortsnamen ausgeben.
***

In [ ]:
multinames = {k:v for k,v in name_idxs.items() if len(v) >1}
multinames

***
Kommt jeder Ortsnamen nur einmal pro Kanton vor?  
Wir haben Gl&uuml;ck, das ist nicht der Fall.
***

In [ ]:
for k,v in multinames.items():
    print(k, [lines[i+1][5] for i in v])

***
Ortsnamen durch anh&auml;ngen des Kantonk&uuml;rzels eindeutig machen.  
Dictionary `name_idx` erstellen, der einem Ortsnamen den Index des zugeh&ouml;rigen Dictionaries in `dictlist` zuordnet.
***

In [ ]:
name_idx = {}
for k,v in name_idxs.items():
    if len(v) == 1:
        name_idx[k] = v[0]
    else:    
        for i in v:
            kt = lines[i+1][5]
            new_key = k + ' ' + kt_kk[kt]
            name_idx[new_key] = i
            citynames[i] = citynames[i] + ' ' + kt_kk[kt]
            
# Auszug aus name_idx
sorted(name_idx.items())[8:15]            

In [ ]:
def citydata(name):
    i = name_idx.get(name.upper())
    if i is None:
        return (None, {})
    return citynames[i], dictlist[i]

citydata('bern'), citydata('tiefencastel')

***
Spezielle Zeichen in Ortsnamen suchen
***

In [ ]:
ABC = ''.join([chr(ord('A')+i) for i in range(26)])
special_chars = {}

for city in name_idx:
    for ch in city:
        if ch not in ABC:
            special_chars[ch] = ord(ch)

special_chars     

In [ ]:
# Doppelnamen
[city for city in name_idx.keys() if '/' in city]

In [ ]:
# Ortsnamen mit ' '
[city for city in name_idx.keys() if ' ' in city]

***
Alternative Ortsnamen hinzuf&uuml;gen, Tippfehler korrigieren
***

In [ ]:
typos        = {'Gross Höchstetten': 'Grosshöchstetten'}
alternatives = {'Konolfingen Dorf': 'Konolfingen', 'Schinznach Dorf': 'Schinznach', 
                'Lucerne': 'Luzern', 'Geneva': 'Genève'}

for k,v in typos.items():
    i = name_idx[k.upper()]
    citynames[i] = v
    name_idx[v.upper()] = name_idx.pop(k.upper())
        
for k,v in alternatives.items():
    name_idx[v.upper()] = name_idx[k.upper()]

In [ ]:
# Ortsnamen mit '’'
[city for city in name_idx.keys() if '’' in city]

In [ ]:
# Ortsnamen mit 'Œ'
[city for city in name_idx.keys() if 'Œ' in city]

***
Doppelnamen 
***

In [ ]:
doublenames = [city for city in name_idx.keys() if '/' in city]
doublenames

***
F&uuml;r jeden Doppelnamen wie Biel/Bienne, Biel und Bienne als Schl&uuml;ssel zu 
`name_idx` hinzuf&uuml;gen
***

In [ ]:
for dname in doublenames:
    idx = name_idx[dname]
    names = dname.split('/')
    for name in names:
        name_idx[name] = idx 

In [ ]:
citydata('biel'), citydata('bienne'), citydata('biel/bienne')

In [ ]:
citydata('domat'), citydata('ems'), citydata('domat/ems')

***
Alternative Namen ohne Umlaute und Akzente hinzuf&uuml;gen 
***

In [ ]:
substitutions = {'Ä': 'AE',
                 'Ö': 'OE', 
                 'Ü': 'UE',
                 'Â': 'A',
                 'Ô': 'O',
                 'É': 'E',
                 'Ê': 'E',
                 'È': 'E',
                 '’': '\'',
                 'Œ': 'OE',
}    

# WENN UEBER EIN OBJEKT ITERIERT WIRD SOLLTE DAS OBJEKT NICHT VERAENDERT WERDEN!
# kann sich nicht darauf verlassen, dass Python Fehlermeldung generiert

# for city in name_idx.keys():
#     idx = name_idx[dn]
#     for k,v in substitutions.items():
#         city = city.replace(k, v)
#     name_idx[city] = idx     



names = list(name_idx)
for name in names:
    idx = name_idx[name]
    for k,v in substitutions.items():
        name = name.replace(k, v)
    name_idx[name] = idx     

In [ ]:
citydata('vandoeuvres')

***
Daten als .pkl-File abspeichern.  
Wird im Modul `ortschaften` verwendet.
***

In [ ]:
import pickle
citydata = (citynames, dictlist, name_idx)

with open('citydata.pkl', mode = 'wb') as f:
    pickle.dump(citydata, f)

In [ ]:
import ortschaften_ch as cities
for name in ['zuerich', 'vandoeuvres', 'bienne', 'Aesch ZH', 'Grosshoechstetten', 'foo']:
    print(cities.cityname(name), 
          cities.coordinates(name), 
          cities.population(name)
         )